The first organizational tool programmers use in Python is the `function`. Functions enable us to break large programs into smaller, simpler pieces. They improve readability and make code more approachable. They allow for reuse and refactoring.

Functions in Python have a variety of extra features that make the programmer's life easier, some of which are uniqure to Python.

## Item 14: Prefer Exceptions to Returning None

See the return value of the function below:

In [2]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None
    
# call the func
x, y = 1, 0
result = divide(x, y)
if not result:
    print('Invalid inputs') # wrong

Invalid inputs


It seems natural to return `None` because the result is undefined. Then when we check the result, a common mistake occurs.

This is why returning `None` is error prone. (`None` and other values, e.g. 0, '', all evaluate to `False` in conditional expressions). see the updated version:

In [3]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs') from e
        
x, y = 5, 2
try:
    result = divide(x, y)
except ValueError:
    print('Invalid inputs')
else:
    print('Result is %.1f' % result)

Result is 2.5


Raise exceptions to indicate special situations instead of returning `None`. Expect the calling code to handle exceptions properly when they're documented.

## Item 15: Know How Closures Interact with Variable Scope

Say you want to sort a list of numbers but prioritize one group of numbers to come first. This pattern is useful when you’re rendering a user interface and want important messages or exceptional events to be displayed before everything else.

A common way to do this is to pass a helper function as the `key` argument to a list’s `sort` method.

In [4]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5 ,7}
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


There are 3 reasons why this func works:
* Python support `closures`: functions that refer to variables from the scope in which they were defined. (helper accesses group argument)
* Functions are `first-class` objects in Python.
* Python has specific rules for comparing tuples.

It'd be nice if this func returned whether higher-priority items were seen at all so the UI code can act accordingly.

In [6]:
def sort_priority2(values, group):
    found = False # scope: sort_priority2
    def helper(x):
        if x in group:
            found = True # scope: helper
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    return found
    
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5 ,7}
found = sort_priority2(numbers, group)
print('Found higher priority items: ', found)
print(numbers)

Found higher priority items:  False
[2, 3, 5, 7, 1, 4, 6, 8]


The result is `False`! How could this be happen?

**When we reference a var in an exp**, the Python interpreter will traverse the scope to resolve the reference in this order:

1. The current func's scope
2. Any closing scopes
3. The scope of the module that contains the code (global scope)
4. The built-in scope (like `len` and `str`)

Otherwise, a `NameError` exception is raised.

**Assigning a value to a var works differently.** If the var doesn't exist in the current scope, Python treats the assignment as a var def.

### Getting Data Out

In Python 3, there is special syntax for getting data out of closure: the `nonlocal` statement.

In [7]:
def sort_priority3(values, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    return found
    
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5 ,7}
found = sort_priority3(numbers, group)
print('Found higher priority items: ', found)
print(numbers)

Found higher priority items:  True
[2, 3, 5, 7, 1, 4, 6, 8]


The `nonlocal` statement makes it clear when data is being assigned out of a closure into another scope. When your usage of nonlocal starts getting complicated, it's better to wrap your state in a helper class.

In [8]:
class Sorter(object):
    def __init__(self, group):
        self.group = group
        self.found = False
        
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)
    
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5 ,7}
sorter = Sorter(group)
numbers.sort(key=sorter)
print('Found higher priority items: ', sorter.found)
print(numbers)

Found higher priority items:  True
[2, 3, 5, 7, 1, 4, 6, 8]


## Item 16: Consider Generators Instead of Returning Lists

The simplest choice for funcs that produce a seq of results is to return a list of items.

In [9]:
def index_words(text):
    result = []
    if text:
        result.append(0)
    for index, letter in enumerate(text):
        if letter == ' ':
            result.append(index+1)
    return result

address = 'Four score and seven years ago...'
result = index_words(address)
print(result[:3])

[0, 5, 11]


There are two problems with the `index_words` function.
1. **the code is a bit dense and noisy**. A better way to write this func is using a `generator`. When called, generator funcs do not actually run but instead immediately return an iterator.
2. **the func requires all results to be stored in the list before being returned**, for huge inputs, this can cause your program to run out of memory and crash.

In [11]:
# problem 1
def index_words(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index+1

address = 'Four score and seven years ago...'
result = list(index_words(address))
print(result[:3])

[0, 5, 11]


It's much clearer because we don't need to maintain the list container manually.

In [21]:
from itertools import islice

# problem 2
def index_file(handle):
    offset = 0
    for line in handle:
        if line:
            yield offset
        for letter in line:
            offset += 1
            if letter == ' ':
                yield offset
                
with open('address.txt', 'r') as f:
    it = index_file(f)
    results = islice(it, 0, 6)
    print(list(results))

[0, 5, 11, 15, 21, 27]


## Item 17: Be Defensive When Iterating Over Arguments

## Item 18: Reduce Visual Noise with Variable Positional Arguments

Accepting optinal positional arguments (often call *star args* in reference to the conventional name for the parameter, `*args`) can make a func call more clear and remove *visual noise*.

Say you have a logging func:

In [24]:
def log(msg, values):
    if not values:
        print(msg)
    else:
        values_str = ','.join(str(x) for x in values)
        print('%s: %s' %(msg, values_str))
        
log('My numbers are', [1, 2])
log('Hi there', [])

My numbers are: 1,2
Hi there


Having to pass an empty list when you have no values to log is **cumbersome and noisy**. So update it to:

In [29]:
def log(msg, *values):
    if not values:
        print(msg)
    else:
        values_str = ','.join(str(x) for x in values)
        print('%s: %s' %(msg, values_str))
        
log('My numbers are', 1, 2)
log('Hi there')

# you already have a list
log('My numbers are', [1, 2])
log('My numbers are', *[1, 2])

My numbers are: 1,2
Hi there
My numbers are: [1, 2]
My numbers are: 1,2


Whether you values to pass, it looks more natural. Note the diff between the 3rd and 4th calling.

Issues of star args
* the variable arguments are always turned into a tuple before they are passed to your function. So for large generator, it's not good choice.
* with `*args`, we can't add new positional arguments to you func in the future without migrating every caller.

For the second issue, use keyword arguments.

## Item 19: Provide Optional Behavior with Keyword Arguments

All positional arguments to Python functions can also be passed by keyword.

Positional arguments must be specified before keyword arguments.

In [31]:
def remainder(number, divisor):
    return number % divisor

print(remainder(20, 7))
print(remainder(20, divisor=7))
print(remainder(number=20, divisor=7))
print(remainder(divisor=7, number=20))

6
6
6
6


Three benefits
* keyword args make the function call clearer to new readers
* they can have default values in func def
* they provide a powerful way to extend a func's params while remaining backwards compatible with existing callers.

A best practice is to always specify optinal arguments using the keyword names and never pass them as positional arguments.

## Item 20: Use `None` and Docstrings to Specify Dynamic Default Arguments

Sometimes you need to use a non-static type as a keyword argument's default value.

In [32]:
from datetime import datetime
from time import sleep

def log(msg, when=datetime.now()):
    print('%s: %s' % (when, msg))
    
log('Hi there!')
sleep(1)
log('Hi again!')


2016-06-08 15:23:51.716590: Hi there!
2016-06-08 15:23:51.716590: Hi again!


The timestamps are the same because `datetime.now` is only executed a single time: when it's defined. **Default arg values are evaluated only once per module load**.

The convention for achieving the desired result in Python is to provide a default value of `None` and to document the actual behavior in the docstring.

In [33]:
def log(msg, when=None):
    """Log a message with a timestamp
    
    Args:
        msg: message to print.
        when: datetime of when the msg occurred.
            defaults to the present time.
    """
    when = datetime.now() if when is None else when
    print('%s: %s' % (when, msg))
    
log('Hi there!')
sleep(1)
log('Hi again!')

2016-06-08 15:28:06.695625: Hi there!
2016-06-08 15:28:07.699474: Hi again!


In [4]:
import json

# dangerous behavior
def decode(data, default={}):
    try:
        return json.loads(data)
    except ValueError:
        return default
    
foo = decode('bad data')
foo['stuff'] = 5

bar = decode('also bad')
bar['meep'] = 1

print('foo:', foo)
print('bar:', bar)
assert foo is bar

foo: {'meep': 1, 'stuff': 5}
bar: {'meep': 1, 'stuff': 5}


## Item 21: Enforce Clarity with Keyword-Only Arguments



In [5]:
def safe_division(number, divisor, 
                  ignore_overflow=False,
                  ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise
            
print(safe_division(1, 10**500, ignore_overflow=True))
print(safe_division(1, 0, ignore_zero_division=True))

0.0
inf


The problem is, since these keyword arguments are optional behavior, there's nothing forcing callers to use keyword args for clarity. See the new `*` symbol:

In [7]:
# python 3
def safe_division(number, divisor, *,
                  ignore_overflow=False,
                  ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise
            
print(safe_division(1, 10**500, ignore_overflow=True))
print(safe_division(1, 0, ignore_zero_division=True))

0.0
inf


In [8]:
# python 2
def print_args(*args, **kwargs):
    print('Positional:', args)
    print('Keyword:', kwargs)
    
print_args(1, 2, foo='bar', stuff='meep')

Positional: (1, 2)
Keyword: {'foo': 'bar', 'stuff': 'meep'}


In [11]:
# python 2
def safe_division2(number, divisor, **kwargs):
    
    ignore_overflow = kwargs.pop('ignore_overflow', False)
    ignore_zero_division = kwargs.pop('ignore_zero_division', False)
    
    if kwargs:
        raise TypeError('Unexpected **kwargs: %r' % kwargs)
    
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise
            
print(safe_division(20, 7))
print(safe_division(1, 10**500, ignore_overflow=True))
print(safe_division(1, 0, ignore_zero_division=True))

2.857142857142857
0.0
inf


In [14]:
# TypeError
#safe_division2(1, 0, False, True)

# TypeError: Unexpected **kwargs: {'unexpected': True}
#safe_division2(0, 0, unexpected=True)